In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import torch

In [3]:
# Settings
SEED = 1
TRAIN_BATCH_SIZE = 1 # on-line learning
TEST_BATCH_SIZE = 128
NUM_WORKERS = 4
LR = 1e-3

# Model hyper-parameters
INPUT_DIM = 7
TF_DROPOUT = 0.0
D_MODEL = 64
N_HEAD = 8
NUM_LAYERS = 4
DIM_FF = 128

# Dataset hyper-parameters
WINDOW_SIZE_MS=100
STRIDE_MS=50

In [4]:
import torch
from torch.utils.data import DataLoader
from src import LazyWindowedDataset, train_test_split

# Set generator for reproducibility
generator = torch.Generator()
generator.manual_seed(SEED)
torch.manual_seed(generator.initial_seed())

# Initialize Dataset
full_train_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="train",
    anomaly_type=['normal'],
    domain_type=['source', 'target'],
    window_size_ms=WINDOW_SIZE_MS,
    stride_ms=STRIDE_MS,
)

train_source_dataset, val_source_dataset = train_test_split(full_train_source_dataset)

train_loader = DataLoader(train_source_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
val_loader = DataLoader(val_source_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, drop_last=True)

test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/RoboticArm",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=WINDOW_SIZE_MS,
    stride_ms=STRIDE_MS,
)
test_loader = DataLoader(test_source_dataset, batch_size=TEST_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)

In [5]:
from models.transformer import RoPeTimeSeriesTransformer

config = {
    'input_dim': INPUT_DIM,
    'd_model': D_MODEL,
    'nhead': N_HEAD,
    'num_layers': NUM_LAYERS,
    'dim_feedforward': DIM_FF,
    'dropout': TF_DROPOUT,
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RoPeTimeSeriesTransformer.from_config(config).to(device)
optimizer = torch.optim.AdamW(model.parameters())
criterior = torch.nn.MSELoss()

In [6]:
from torchinfo import summary
summary(model, input_size=(TRAIN_BATCH_SIZE, 1600, 7))

Layer (type:depth-idx)                        Output Shape              Param #
RoPeTimeSeriesTransformer                     [1, 1600, 7]              --
├─HybridSensorPositionalEncoding: 1-1         [1, 1600, 7]              --
├─Linear: 1-2                                 [1, 1600, 64]             512
├─TransformerEncoder: 1-3                     [1, 1600, 64]             --
│    └─ModuleList: 2-1                        --                        --
│    │    └─TransformerEncoderLayer: 3-1      [1, 1600, 64]             33,472
│    │    └─TransformerEncoderLayer: 3-2      [1, 1600, 64]             33,472
│    │    └─TransformerEncoderLayer: 3-3      [1, 1600, 64]             33,472
│    │    └─TransformerEncoderLayer: 3-4      [1, 1600, 64]             33,472
├─Linear: 1-4                                 [1, 1600, 7]              455
Total params: 134,855
Trainable params: 134,855
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.07
Input size (MB): 0.04
Forward/backward p

In [7]:
from src import train_model, evaluate

try:
    train_model(
        name='ra_aerodactyl', 
        model=model, 
        criterion=criterior, 
        optimizer=optimizer, 
        train_loader=train_loader, 
        val_loader=val_loader, # Skip validation to speed up
        merge_startegy='stack',
        num_epochs=1, 
        verbose=1,
        train_num_batches=50, # Do not train on the whole epoch, but some random choosen batches
        val_num_batches=50, # Do not validate on the whole epoch, but some random choosen batches
        save_every=1,
        generator=generator,
    )
except KeyboardInterrupt:
    print("Training interrupted by user.")

Evaluation: 100%|██████████| 50/50 [00:01<00:00, 41.72batch/s]

Epoch [1/1] (Checkpoint Epoch: 1) | Train Loss: 0.824327 | Val Loss: 0.760491 | Val AUC: nan
Time Spent: 3.85s | ETA: 0.00s | Current Time: 2025-05-22 11:15:56
Checkpoint saved at checkpoints/ra_aerodactyl_RoPeTimeSeriesTransformer_epoch_1.pt


In [8]:
loss, auc = evaluate(model, test_loader, criterior, merge_strategy='stack', verbose=True, generator=generator)
print(f"Overall S+T | Loss: {loss:.4f}, AUC: {auc:.4f}")

Evaluation: 100%|██████████| 540/540 [05:43<00:00,  1.57batch/s]

Overall S+T | Loss: 0.7622, AUC: 0.8313
